In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
games_df = pd.read_csv('espn_data/raw_data/2026.csv',index_col=0)
games_df.head()

,date,game_id,away,home,neutral,minutes,away_points,away_fga,away_fta,away_tov,...,home_points,home_fga,home_fta,home_tov,home_oreb,time_et,location,ref1,ref2,ref3
0,2025-11-03,401824809,Lehigh,Houston,0,40.0,57,51,23,15,...,75,61,20,13,15,8:00 PM,"Houston,TX",Nomi Kidwai,Tommy Johnson,De Selle Dean
0,2025-11-03,401826885,Florida,Arizona,1,40.0,87,70,30,15,...,93,61,38,14,12,7:00 PM,"Las Vegas,NV",Kipp Kissinger,Keith Kimble,Bart Lenox
0,2025-11-03,401812785,New Haven,UConn,0,40.0,55,52,14,11,...,79,56,20,12,10,7:00 PM,"Storrs,CT",Jeff Clark,Tim Clougherty,Vasili Mallios
0,2025-11-03,401820577,Quinnipiac,St. John's,0,40.0,74,67,19,22,...,108,69,34,14,16,6:30 PM,"Queens,NY",Nathan Farrell,Michael Palau,Tim Smith
0,2025-11-03,401826083,Oakland,Michigan,0,40.0,78,64,15,11,...,121,70,13,8,6,8:30 PM,"Ann Arbor,MI",Brian McNutt,Jeffrey Anderson,Anthony Burris


In [5]:
# adjust for bad minutes played data
games_df['minutes'] = games_df['minutes'].apply(lambda x: max(40, x))

In [6]:
games_df['date'] = games_df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [7]:
games_df['away_poss'] = games_df['away_fga'] - games_df['away_oreb'] + games_df['away_tov'] + 0.475*games_df['away_fta']
games_df['home_poss'] = games_df['home_fga'] - games_df['home_oreb'] + games_df['home_tov'] + 0.475*games_df['home_fta']
games_df['away_ppp'] = games_df['away_points'] / games_df['away_poss']
games_df['home_ppp'] = games_df['home_points'] / games_df['home_poss']
games_df['away_tempo'] = games_df['away_poss'] / games_df['minutes']
games_df['home_tempo'] = games_df['home_poss'] / games_df['minutes']
games_df['game_tempo'] = (games_df['away_tempo'] + games_df['home_tempo'])/2
games_df.head()

,date,game_id,away,home,neutral,minutes,away_points,away_fga,away_fta,away_tov,...,ref1,ref2,ref3,away_poss,home_poss,away_ppp,home_ppp,away_tempo,home_tempo,game_tempo
0,2025-11-03,401824809,Lehigh,Houston,0,40.0,57,51,23,15,...,Nomi Kidwai,Tommy Johnson,De Selle Dean,67.925,68.500,0.839161,1.094891,1.698125,1.712500,1.705312
0,2025-11-03,401826885,Florida,Arizona,1,40.0,87,70,30,15,...,Kipp Kissinger,Keith Kimble,Bart Lenox,82.250,81.050,1.057751,1.147440,2.056250,2.026250,2.041250
0,2025-11-03,401812785,New Haven,UConn,0,40.0,55,52,14,11,...,Jeff Clark,Tim Clougherty,Vasili Mallios,65.650,67.500,0.837776,1.170370,1.641250,1.687500,1.664375
0,2025-11-03,401820577,Quinnipiac,St. John's,0,40.0,74,67,19,22,...,Nathan Farrell,Michael Palau,Tim Smith,84.025,83.150,0.880690,1.298857,2.100625,2.078750,2.089688
0,2025-11-03,401826083,Oakland,Michigan,0,40.0,78,64,15,11,...,Brian McNutt,Jeffrey Anderson,Anthony Burris,77.125,78.175,1.011345,1.547809,1.928125,1.954375,1.941250


In [8]:
team_counts = pd.concat([games_df['away'], games_df['home']]).value_counts().to_dict()
dict(sorted(team_counts.items(), key=lambda item: item[1]))

{'Arkansas Tech': 1,
 'Alfred State': 1,
 'Bryant & Stratton (Ohio)': 1,
 'Arcadia': 1,
 'Georgia College': 1,
 'Pacific Union': 1,
 'McPherson': 1,
 'Mid-America Christian': 1,
 'Truett-McConnell': 1,
 'Penn State-Behrend': 1,
 'Elmira': 1,
 'Blackburn': 1,
 'WVU Tech': 1,
 'Franciscan University': 1,
 'Sarah Lawrence': 1,
 'Williams Baptist': 1,
 'Emerson': 1,
 'Morehouse': 1,
 'Earlham College': 1,
 'Principia College': 1,
 'Concord': 1,
 'Mobile': 1,
 'Shenandoah': 1,
 'North Central (MN)': 1,
 'Redlands': 1,
 'Alice Lloyd College': 1,
 'Benedictine (AZ)': 1,
 'Baptist (Fl)': 1,
 'Regis (CO)': 1,
 'Our Lady Of The Lake': 1,
 'Centenary (LA)': 1,
 'Maine-Fort Kent': 1,
 'SUNY-Canton': 1,
 'Mitchell College': 1,
 'Gwynedd-Mercy': 1,
 'Northeastern State': 1,
 'University of The Southwest': 1,
 'Lawrence University': 1,
 'St. Ambrose': 1,
 'Kentucky State': 1,
 'Crown College': 1,
 'Louisiana Christian': 1,
 'Florida Memorial': 1,
 'George Fox': 1,
 'Simpson University (CA)': 1,
 'Spu

In [9]:
# remove all games with d2 teams
games_df = games_df[
    games_df['away'].map(team_counts).ge(16) &
    games_df['home'].map(team_counts).ge(16)]
team_counts = pd.concat([games_df['away'], games_df['home']]).value_counts().to_dict()

In [11]:
# date range from start to end of csv
start_date = datetime(2025,12,15)
end_date = datetime(2026,1,22)
dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += timedelta(days=1)

In [12]:
import math

def measure_distance(d1, d2):
    return sum(abs(d1[k] - d2[k]) for k in d1)

def calc_adj_tempo(game_tempo, opp_tempo):
    return 2 * game_tempo - opp_tempo

def margin_decay(x, decay=30):
    #return min(1, math.exp(-((x-40)/decay)))
    return 1

def exp_decay(x, decay=90):
    return math.exp(-(x/decay))

In [14]:
# iterate to calculate adjusted net ratings
def calc_ratings(df, mean_oe, team_counts, oe_dict, de_dict, present_date, hca=0.02):
    iteration = 1
    while iteration < 30:
    
        oe_iter_dict = dict.fromkeys(keys, 0)
        de_iter_dict = dict.fromkeys(keys, 0)
        oe_counts_dict = dict.fromkeys(keys, 0)
        de_counts_dict = dict.fromkeys(keys, 0)
        
        
        for row in games_df.itertuples():
            away = row.away
            home = row.home

            away_ppp = row.away_ppp
            home_ppp = row.home_ppp

            neutral = row.neutral

            if not neutral:
                away_ppp += hca
                home_ppp -= hca
        

            away_adj_oe = (away_ppp - mean_oe) - (de_dict[home] - mean_oe) + mean_oe
            home_adj_oe = (home_ppp - mean_oe) - (de_dict[away] - mean_oe) + mean_oe
        
            away_adj_de = (home_ppp - mean_oe) - (oe_dict[home] - mean_oe) + mean_oe
            home_adj_de = (away_ppp - mean_oe) - (oe_dict[away] - mean_oe) + mean_oe


            margin_weight = margin_decay(abs(row.away_points - row.home_points))

            days_ago = (present_date - row.date).days
            recency_weight = exp_decay(days_ago)
        
            oe_iter_dict[away] += margin_weight*recency_weight*row.away_poss*away_adj_oe
            oe_iter_dict[home] += margin_weight*recency_weight*row.home_poss*home_adj_oe
        
            de_iter_dict[away] += margin_weight*recency_weight*row.home_poss*away_adj_de
            de_iter_dict[home] += recency_weight*row.away_poss*home_adj_de
        
            oe_counts_dict[away] += margin_weight*recency_weight*row.away_poss
            oe_counts_dict[home] += margin_weight*recency_weight*row.home_poss
            de_counts_dict[away] += margin_weight*recency_weight*row.home_poss
            de_counts_dict[home] += margin_weight*recency_weight*row.away_poss
        
        
        
        new_oe_dict = {k: oe_iter_dict[k] / oe_counts_dict[k] for k in oe_iter_dict}
        new_de_dict = {k: de_iter_dict[k] / de_counts_dict[k] for k in de_iter_dict}
        
        
        oe_dist = measure_distance(oe_dict, new_oe_dict)
        de_dist = measure_distance(de_dict, new_de_dict)
        
        iteration += 1
        
        oe_dict = new_oe_dict
        de_dict = new_de_dict

        team_tempo = pd.concat([
            df[['home', 'game_tempo']].rename(columns={'home': 'team'}),
            df[['away', 'game_tempo']].rename(columns={'away': 'team'})
        ])
        tempo_std_dict = (
            team_tempo
            .groupby('team')['game_tempo']
            .std()
            .to_dict()
        )

    return oe_dict, de_dict, tempo_std_dict#, tempo_dict

# least squares to calculate tempo coefficients
def calc_tempo(df, keys, mean_tempo, tempo_dict):
    teams = list(keys)
    idx = {t: i for i, t in enumerate(teams)}
    n = len(teams)
    
    rows = []
    b = []
    
    for row in df.itertuples():
        r = np.zeros(n)
        r[idx[row.away]] = 1
        r[idx[row.home]] = 1
        rows.append(r)
        b.append(row.away_tempo + row.home_tempo)
    
    A = np.vstack(rows)
    b = np.array(b)
    
    # constraint: mean tempo
    C = np.ones((1, n))
    d = np.array([n * mean_tempo])
    
    A = np.vstack([A, C])
    b = np.concatenate([b, d])
    
    T = np.linalg.lstsq(A, b, rcond=None)[0]
    
    tempo_dict = dict(zip(teams, T))

    return tempo_dict


def predict_spread(team1, team2, oe_dict, de_dict, tempo_dict, tempo_std_dict, neutral=0, hca=0.02):
    #tempo = 40*(tempo_dict[team1] + tempo_dict[team2])/2
    tempo = 40*( ((1/tempo_std_dict[team1])*tempo_dict[team1] + (1/tempo_std_dict[team2])*tempo_dict[team2]) / (1/tempo_std_dict[team1] + 1/tempo_std_dict[team2])) 
    if abs(tempo_std_dict[team1] - tempo_std_dict[team2]) > 0.02:
        tempo = 40*( ((1/tempo_std_dict[team1])*tempo_dict[team1] + (1/tempo_std_dict[team2])*tempo_dict[team2]) / (1/tempo_std_dict[team1] + 1/tempo_std_dict[team2])) 
    else:
        tempo = 40*(tempo_dict[team1] + tempo_dict[team2])/2
   
    
    team1_ppp = (oe_dict[team1] - mean_oe) - (mean_oe - de_dict[team2]) + mean_oe
    team2_ppp = (oe_dict[team2] - mean_oe) - (mean_oe - de_dict[team1]) + mean_oe

    if not neutral:
        team1_ppp -= hca
        team2_ppp += hca

    team1_pred = tempo*team1_ppp
    team2_pred = tempo*team2_ppp
    ou = team1_pred + team2_pred

    return team1_pred, team2_pred, ou


# predict final scores
# track error metrics
def predict(df, oe_dict, de_dict, tempo_dict, tempo_std_dict):
    total = 0
    sum_diffs_spread = 0
    sq_sum_diffs_spread = 0

    sum_diffs_ou = 0
    sq_sum_diffs_ou = 0

    for row in df.itertuples():
        away = row.away
        home = row.home

        neutral = row.neutral

        true_spread = row.away_points - row.home_points
        true_ou = row.away_points + row.home_points

        away_pred, home_pred, ou_pred = predict_spread(away, home, oe_dict, de_dict, tempo_dict, tempo_std_dict, neutral)
        spread_pred = away_pred - home_pred

        spread_diff = abs(spread_pred - true_spread)
        ou_diff = abs(ou_pred - true_ou)

        sum_diffs_spread += spread_diff
        sq_sum_diffs_spread += spread_diff**2

        sum_diffs_ou += ou_diff
        sq_sum_diffs_ou += ou_diff**2

        total += 1

    return sum_diffs_spread / total, sq_sum_diffs_spread / total, sum_diffs_ou / total, sq_sum_diffs_ou / total, total


def print_error_summary(error_data):
    denom = 0
    total_spread_mae = 0
    total_ou_mae = 0

    for row in error_data:
        total_spread_mae += row[4] * row[0]
        total_ou_mae += row[4] * row[2]
        denom += row[4]

    full_spread_mae = total_spread_mae / denom
    full_ou_mae = total_ou_mae / denom

    print('---Full Season Error---')
    print(f'Number of games: {denom}')
    print(f'Spread MAE: {full_spread_mae:.2f}')
    print(f'OU MAE: {full_ou_mae:.2f}')

In [15]:
error_data = []
for date in dates:
    fit_df = games_df[games_df.date < date]
    mean_oe = (fit_df.away_ppp.mean() + fit_df.home_ppp.mean()) / 2
    mean_tempo = (fit_df.away_tempo.mean() + fit_df.home_tempo.mean()) / 2
    
    keys = set(games_df['away']).union(games_df['home'])
    
    oe_dict = dict.fromkeys(keys, mean_oe)
    de_dict = dict.fromkeys(keys, mean_oe)
    tempo_dict = dict.fromkeys(keys, mean_tempo)

    team_counts = pd.concat([fit_df['away'], fit_df['home']]).value_counts().to_dict()

    oe_dict, de_dict, tempo_std_dict = calc_ratings(fit_df, mean_oe, team_counts, oe_dict, de_dict, date)
    tempo_dict = calc_tempo(fit_df, keys, mean_tempo, tempo_dict)

    predict_df = games_df[games_df.date == date]

    if len(predict_df) == 0:
        continue

    spread_mae, spread_mse, ou_mae, ou_mse, total = predict(predict_df, oe_dict, de_dict, tempo_dict, tempo_std_dict)

    error_data.append([spread_mae, spread_mse, ou_mae, ou_mse, total, date])

    print(f'Predict Date: {date}')
    print(f'Number of games: {total}')
    print(f'Spread MAE: {spread_mae}')
    #print(f'Spread MSE: {spread_mse}')
    print(f'OU MAE: {ou_mae}')
    #print(f'OU MSE: {ou_mse}\n')

print_error_summary(error_data)

Predict Date: 2025-12-15 00:00:00
Number of games: 7
Spread MAE: 7.634364302154293
OU MAE: 9.579777769429967
Predict Date: 2025-12-16 00:00:00
Number of games: 36
Spread MAE: 7.53163850814058
OU MAE: 11.865392705045423
Predict Date: 2025-12-17 00:00:00
Number of games: 50
Spread MAE: 8.631819334628284
OU MAE: 12.060656397631282
Predict Date: 2025-12-18 00:00:00
Number of games: 27
Spread MAE: 7.324697229885742
OU MAE: 14.426527064358035
Predict Date: 2025-12-19 00:00:00
Number of games: 23
Spread MAE: 9.81579832593493
OU MAE: 10.365259213454522
Predict Date: 2025-12-20 00:00:00
Number of games: 64
Spread MAE: 8.687793046132951
OU MAE: 12.868887541655077
Predict Date: 2025-12-21 00:00:00
Number of games: 52
Spread MAE: 8.588862084938333
OU MAE: 12.162709278256745
Predict Date: 2025-12-22 00:00:00
Number of games: 42
Spread MAE: 9.35296120190527
OU MAE: 11.623635402869295
Predict Date: 2025-12-23 00:00:00
Number of games: 11
Spread MAE: 6.477432191026736
OU MAE: 14.498167115825499
Predic

In [16]:
# predict scores for the NEXT day

date = datetime(2026, 1, 23)
fit_df = games_df[games_df.date < date]
mean_oe = (fit_df.away_ppp.mean() + fit_df.home_ppp.mean()) / 2
mean_tempo = (fit_df.away_tempo.mean() + fit_df.home_tempo.mean()) / 2
#print(mean_tempo)

keys = set(games_df['away']).union(games_df['home'])

oe_dict = dict.fromkeys(keys, mean_oe)
de_dict = dict.fromkeys(keys, mean_oe)
tempo_dict = dict.fromkeys(keys, mean_tempo)

team_counts = pd.concat([fit_df['away'], fit_df['home']]).value_counts().to_dict()

oe_dict, de_dict, tempo_std_dict = calc_ratings(fit_df, mean_oe, team_counts, oe_dict, de_dict, date)
tempo_dict = calc_tempo(fit_df, keys, mean_tempo, tempo_dict)

In [17]:
away_pts, home_pts, ou = predict_spread("Arizona", "Duke", oe_dict, de_dict, tempo_dict, tempo_std_dict)
print(f'{away_pts:.1f}', f'{home_pts:.1f}')
print(f'{away_pts-home_pts:.1f}')
print(f'{ou:.1f}')

76.0 80.2
-4.2
156.2


In [19]:
error_df = pd.DataFrame(error_data)
error_df['month'] = error_df[5].apply(lambda x: x.month)
error_df

,0,1,2,3,4,5,month
0,7.634364,82.210550,9.579778,179.092250,7,2025-12-15,12
1,7.531639,91.378307,11.865393,217.485565,36,2025-12-16,12
2,8.631819,128.565693,12.060656,215.139155,50,2025-12-17,12
3,7.324697,86.121388,14.426527,349.600844,27,2025-12-18,12
4,9.815798,135.102860,10.365259,182.221913,23,2025-12-19,12
5,8.687793,118.062735,12.868888,289.430546,64,2025-12-20,12
6,8.588862,121.742534,12.162709,225.835762,52,2025-12-21,12
7,9.352961,141.124435,11.623635,194.637765,42,2025-12-22,12
8,6.477432,69.893537,14.498167,278.919540,11,2025-12-23,12
9,8.328332,69.361114,0.585941,0.343327,1,2025-12-27,12


In [20]:
error_df.groupby('month').agg({
    0: 'mean',
    2: 'mean'})

,0,2
month,,
1,7.501014,12.450380
12,8.256799,11.520391


In [21]:
dict(sorted(oe_dict.items(), key=lambda item: -item[1]))

{'Illinois': np.float64(1.3003531352512816),
 'Purdue': np.float64(1.2939284000655977),
 'Arkansas': np.float64(1.2865702130668835),
 'Alabama': np.float64(1.2837814667244798),
 'Michigan': np.float64(1.2821272724224044),
 'Duke': np.float64(1.27218179511042),
 'Arizona': np.float64(1.2664097236616276),
 'Louisville': np.float64(1.264686312232711),
 'Iowa State': np.float64(1.2570943888348614),
 'Utah State': np.float64(1.2556944870368154),
 'Texas Tech': np.float64(1.254216765562263),
 'SMU': np.float64(1.2527965073482215),
 'Florida': np.float64(1.2521545472114233),
 'Gonzaga': np.float64(1.2520493924353961),
 'Vanderbilt': np.float64(1.2514533634245253),
 'Texas': np.float64(1.2511151604545447),
 'Tulsa': np.float64(1.250560069556743),
 'Virginia': np.float64(1.2503101466767164),
 'Auburn': np.float64(1.2498743469270588),
 'BYU': np.float64(1.243067678718444),
 'Ohio State': np.float64(1.241305965162773),
 'Houston': np.float64(1.2343187096360562),
 'Baylor': np.float64(1.2318421700

In [22]:
ratings_dict = {k: oe_dict[k] - de_dict[k] for k in oe_dict}
dict(sorted(ratings_dict.items(), key=lambda item: -item[1]))

{'Michigan': np.float64(0.42129156780743593),
 'Duke': np.float64(0.39939808411536115),
 'Arizona': np.float64(0.38198913952964086),
 'Gonzaga': np.float64(0.3581968705338925),
 'Houston': np.float64(0.35585342431815636),
 'Purdue': np.float64(0.35085040821173963),
 'Illinois': np.float64(0.3489375185709591),
 'Iowa State': np.float64(0.3355150862004098),
 'Florida': np.float64(0.330484875905658),
 'Virginia': np.float64(0.32244972603799915),
 'Nebraska': np.float64(0.3222445695677921),
 'Michigan State': np.float64(0.3191172760360298),
 'Louisville': np.float64(0.31527971272208766),
 'UConn': np.float64(0.31018795190130777),
 'Texas Tech': np.float64(0.30475118602294426),
 'BYU': np.float64(0.30018405479168964),
 'Kansas': np.float64(0.29861923407100244),
 'Arkansas': np.float64(0.2932851836195979),
 'Iowa': np.float64(0.28700204633960225),
 'Alabama': np.float64(0.2852277379901782),
 'Vanderbilt': np.float64(0.273631774615004),
 "St. John's": np.float64(0.27328785137461664),
 'Saint 